# 🔤 Tokenization - Transformer le Texte en Nombres

## 🎯 Objectifs

Dans ce notebook, nous allons apprendre :

- 🔢 **Pourquoi tokenizer ?** - Les réseaux ne comprennent que les nombres
- 📚 **Différentes approches** - Character, Word, Subword
- 🧩 **BPE (Byte-Pair Encoding)** - L'algorithme utilisé par GPT
- 🛠️ **Implémenter un tokenizer** - From scratch en Python
- 🎨 **Encoder/Decoder** - Texte ↔ Nombres

---

## 🤔 Pourquoi la Tokenization ?

### Le Problème

Les réseaux de neurones ne comprennent que des **nombres** :

```
❌ "Bonjour le monde"  → ??? Comment donner ça au réseau ?
✅ [145, 298, 1023]     → Le réseau comprend ça !
```

### La Solution : Tokenization

**Tokenization** = Découper le texte en unités (tokens) et les associer à des nombres

```
Texte : "Bonjour le monde"
  ↓ Tokenization
Tokens : ["Bonjour", "le", "monde"]
  ↓ Encoding
IDs : [145, 298, 1023]
```

---

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import re

print("✓ Prêt à tokenizer !")

## 📊 Trois Approches de Tokenization

### 1️⃣ Character-Level (Par caractère)

**Idée** : Chaque caractère = 1 token

```python
Texte : "Bonjour"
Tokens : ['B', 'o', 'n', 'j', 'o', 'u', 'r']
```

**Avantages** :
- ✅ Vocabulaire très petit (~100 caractères)
- ✅ Pas de mots inconnus (OOV)

**Inconvénients** :
- ❌ Séquences très longues
- ❌ Perd la sémantique des mots

### 2️⃣ Word-Level (Par mot)

**Idée** : Chaque mot = 1 token

```python
Texte : "Bonjour le monde"
Tokens : ['Bonjour', 'le', 'monde']
```

**Avantages** :
- ✅ Préserve la sémantique
- ✅ Séquences courtes

**Inconvénients** :
- ❌ Vocabulaire énorme (100k+ mots)
- ❌ Mots inconnus (OOV problem)
- ❌ Ne gère pas bien les variations ("jouer", "jouait", "jouerait")

### 3️⃣ Subword-Level (Par sous-mot) ⭐ **BEST**

**Idée** : Découper en morceaux de mots (subwords)

```python
Texte : "incroyablement"
Tokens : ['in', 'croy', 'able', 'ment']
```

**Avantages** :
- ✅ Vocabulaire raisonnable (30k-50k tokens)
- ✅ Pas de OOV (tout peut être décomposé)
- ✅ Gère bien les variations
- ✅ **C'est ce qu'utilise GPT !**

**Algorithmes** : BPE, WordPiece, SentencePiece

---

In [ ]:
# Exemple des 3 approches

text = "Bonjour le monde"

# 1. Character-level
char_tokens = list(text)
print("1️⃣ Character-level:")
print(f"   Tokens: {char_tokens}")
print(f"   Longueur: {len(char_tokens)} tokens\n")

# 2. Word-level
word_tokens = text.split()
print("2️⃣ Word-level:")
print(f"   Tokens: {word_tokens}")
print(f"   Longueur: {len(word_tokens)} tokens\n")

# 3. Subword-level (simulation)
subword_tokens = ['Bon', 'jour', 'le', 'monde']  # Exemple
print("3️⃣ Subword-level (simulé):")
print(f"   Tokens: {subword_tokens}")
print(f"   Longueur: {len(subword_tokens)} tokens")

print("\n💡 Subword = Meilleur compromis !")

---

## 🧩 BPE - Byte-Pair Encoding

### Concept

**BPE** est un algorithme de compression adapté pour la tokenization :

1. Commence avec un vocabulaire de caractères
2. Trouve la paire de tokens la plus fréquente
3. Fusionne cette paire en un nouveau token
4. Répète jusqu'à atteindre la taille de vocabulaire désirée

### Exemple Pas à Pas

```
Texte initial : "low low low low lower lower newest newest newest widest"

Étape 0 (caractères) :
Vocab : ['l', 'o', 'w', 'e', 'r', 'n', 's', 't', 'i', 'd']
Tokens : ['l','o','w', 'l','o','w', ...]

Étape 1 - Paire la plus fréquente : ('l', 'o')
Nouveau token : 'lo'
Vocab : ['l', 'o', 'w', 'e', 'r', 'n', 's', 't', 'i', 'd', 'lo']
Tokens : ['lo','w', 'lo','w', ...]

Étape 2 - Paire la plus fréquente : ('lo', 'w')
Nouveau token : 'low'
Vocab : [..., 'lo', 'low']
Tokens : ['low', 'low', 'low', 'low', 'lo','w','e','r', ...]

Étape 3 - Paire la plus fréquente : ('e', 's')
Nouveau token : 'es'
...
```

**Résultat** : Vocabulaire optimisé pour le corpus !

---

## 🛠️ Implémentation d'un Tokenizer BPE Simple

On va implémenter un tokenizer BPE from scratch !

In [ ]:
class SimpleBPETokenizer:
    """
    Tokenizer BPE simplifié pour l'apprentissage
    """

    def __init__(self, vocab_size=256):
        self.vocab_size = vocab_size
        self.vocab = {}  # token → id
        self.inverse_vocab = {}  # id → token
        self.merges = {}  # paire → nouveau_token

    def get_pairs(self, word):
        """
        Récupère toutes les paires consécutives dans un mot
        
        Exemple : ['l', 'o', 'w'] → [('l', 'o'), ('o', 'w')]
        """
        pairs = set()
        prev_char = word[0]
        for char in word[1:]:
            pairs.add((prev_char, char))
            prev_char = char
        return pairs

    def train(self, corpus, verbose=True):
        """
        Entraîne le tokenizer BPE sur un corpus
        
        Args:
            corpus: Liste de mots
            verbose: Afficher les étapes
        """
        # 1. Initialiser avec les caractères
        vocab = set()
        for word in corpus:
            vocab.update(word)

        # Créer le vocabulaire initial (caractères)
        self.vocab = {char: idx for idx, char in enumerate(sorted(vocab))}
        self.inverse_vocab = {idx: char for char, idx in self.vocab.items()}

        if verbose:
            print(f"Vocabulaire initial: {len(self.vocab)} caractères")
            print(f"Objectif: {self.vocab_size} tokens\n")

        # 2. Répéter les merges jusqu'à atteindre vocab_size
        word_freqs = Counter(corpus)
        splits = {word: list(word) for word in word_freqs.keys()}

        num_merges = self.vocab_size - len(self.vocab)

        for i in range(num_merges):
            # Compter toutes les paires
            pairs = defaultdict(int)
            for word, freq in word_freqs.items():
                split = splits[word]
                if len(split) == 1:
                    continue
                for j in range(len(split) - 1):
                    pair = (split[j], split[j + 1])
                    pairs[pair] += freq

            if not pairs:
                break

            # Trouver la paire la plus fréquente
            best_pair = max(pairs, key=pairs.get)
            
            # Créer le nouveau token
            new_token = best_pair[0] + best_pair[1]
            self.merges[best_pair] = new_token

            # Ajouter au vocabulaire
            new_id = len(self.vocab)
            self.vocab[new_token] = new_id
            self.inverse_vocab[new_id] = new_token

            # Mettre à jour les splits
            for word in word_freqs:
                split = splits[word]
                if len(split) == 1:
                    continue

                new_split = []
                j = 0
                while j < len(split):
                    if j < len(split) - 1 and (split[j], split[j + 1]) == best_pair:
                        new_split.append(new_token)
                        j += 2
                    else:
                        new_split.append(split[j])
                        j += 1
                splits[word] = new_split

            if verbose and (i + 1) % 10 == 0:
                print(f"Merge {i+1}/{num_merges}: {best_pair} → '{new_token}' (freq: {pairs[best_pair]})")

        if verbose:
            print(f"\n✓ Entraînement terminé ! Vocabulaire final: {len(self.vocab)} tokens")

    def encode(self, text):
        """
        Encode un texte en liste d'IDs
        
        Args:
            text: Texte à encoder
        
        Returns:
            Liste d'IDs de tokens
        """
        # Tokenize d'abord en caractères
        tokens = list(text)

        # Appliquer les merges
        while len(tokens) >= 2:
            pairs = [(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)]
            
            # Trouver la première paire qui a un merge
            merge_found = False
            for i, pair in enumerate(pairs):
                if pair in self.merges:
                    # Appliquer le merge
                    new_token = self.merges[pair]
                    tokens = tokens[:i] + [new_token] + tokens[i + 2:]
                    merge_found = True
                    break
            
            if not merge_found:
                break

        # Convertir en IDs
        ids = []
        for token in tokens:
            if token in self.vocab:
                ids.append(self.vocab[token])
            else:
                # Token inconnu → utiliser un caractère spécial ou ignorer
                ids.append(0)  # <UNK>

        return ids

    def decode(self, ids):
        """
        Décode une liste d'IDs en texte
        
        Args:
            ids: Liste d'IDs de tokens
        
        Returns:
            Texte décodé
        """
        tokens = [self.inverse_vocab.get(id, '<UNK>') for id in ids]
        return ''.join(tokens)

print("✓ Classe SimpleBPETokenizer définie")

### Test du Tokenizer BPE

In [ ]:
# Corpus d'exemple
corpus = [
    "low", "low", "low", "low",
    "lower", "lower",
    "newest", "newest", "newest",
    "widest"
]

print("📚 Corpus d'entraînement:")
print(f"   {' '.join(corpus)}")
print(f"   {len(corpus)} mots\n")

# Créer et entraîner le tokenizer
tokenizer = SimpleBPETokenizer(vocab_size=30)
tokenizer.train(corpus, verbose=True)

In [ ]:
# Afficher le vocabulaire appris
print("\n📖 Vocabulaire appris:")
print("─" * 50)

# Grouper par taille de token
by_length = defaultdict(list)
for token, idx in sorted(tokenizer.vocab.items(), key=lambda x: x[1]):
    by_length[len(token)].append(token)

for length in sorted(by_length.keys()):
    tokens = by_length[length]
    print(f"\nLongueur {length}: {tokens}")

print(f"\n📊 Total: {len(tokenizer.vocab)} tokens")

In [ ]:
# Tester l'encodage/décodage
test_texts = [
    "low",
    "lower",
    "newest",
    "lowest"  # Nouveau mot !
]

print("\n🧪 Test Encode/Decode:")
print("─" * 70)

for text in test_texts:
    # Encoder
    ids = tokenizer.encode(text)
    
    # Décoder
    decoded = tokenizer.decode(ids)
    
    # Afficher les tokens
    tokens = [tokenizer.inverse_vocab[id] for id in ids]
    
    print(f"\nTexte:   '{text}'")
    print(f"Tokens:  {tokens}")
    print(f"IDs:     {ids}")
    print(f"Décodé:  '{decoded}'")
    print(f"Match:   {'✓' if text == decoded else '✗'}")

---

## 🎨 Visualisation du Processus de Tokenization

In [ ]:
# Visualiser la distribution des longueurs de tokens
import matplotlib.pyplot as plt

token_lengths = [len(token) for token in tokenizer.vocab.keys()]
length_counts = Counter(token_lengths)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme des longueurs
lengths = sorted(length_counts.keys())
counts = [length_counts[l] for l in lengths]

ax1.bar(lengths, counts, color='steelblue', edgecolor='black', linewidth=1.5)
ax1.set_xlabel('Longueur du Token', fontsize=12, fontweight='bold')
ax1.set_ylabel('Nombre de Tokens', fontsize=12, fontweight='bold')
ax1.set_title('Distribution des Longueurs de Tokens', fontsize=14, fontweight='bold')
ax1.grid(axis='y', alpha=0.3)

# Ajouter les valeurs
for l, c in zip(lengths, counts):
    ax1.text(l, c + 0.2, str(c), ha='center', fontweight='bold')

# Exemple de tokenization
example = "lowest"
example_ids = tokenizer.encode(example)
example_tokens = [tokenizer.inverse_vocab[id] for id in example_ids]

ax2.axis('off')
ax2.text(0.5, 0.9, f'Exemple: "{example}"', 
         ha='center', fontsize=16, fontweight='bold', transform=ax2.transAxes)

# Afficher la décomposition
y = 0.7
for i, (token, tid) in enumerate(zip(example_tokens, example_ids)):
    color = plt.cm.Set3(i % 10)
    
    ax2.add_patch(plt.Rectangle((0.2 + i*0.15, y - 0.05), 0.12, 0.08, 
                                 facecolor=color, edgecolor='black', linewidth=2,
                                 transform=ax2.transAxes))
    
    ax2.text(0.26 + i*0.15, y, f'"{token}"', 
             ha='center', va='center', fontsize=14, fontweight='bold',
             transform=ax2.transAxes)
    
    ax2.text(0.26 + i*0.15, y - 0.15, f'ID: {tid}', 
             ha='center', va='center', fontsize=10,
             transform=ax2.transAxes)

ax2.text(0.5, 0.3, 'Les tokens BPE capturent les sous-mots fréquents !', 
         ha='center', fontsize=12, style='italic', transform=ax2.transAxes)

plt.tight_layout()
plt.show()

---

## 🌍 Tokenizers Réels : GPT-2, GPT-3, GPT-4

### Spécifications

| Modèle | Algorithme | Vocab Size | Tokens Spéciaux |
|--------|-----------|------------|------------------|
| **GPT-2** | BPE | 50,257 | `<|endoftext|>` |
| **GPT-3** | BPE | 50,257 | `<|endoftext|>` |
| **GPT-4** | BPE amélioré | ~100,000 | Multiples |
| **BERT** | WordPiece | 30,522 | `[CLS]`, `[SEP]`, `[MASK]` |
| **T5** | SentencePiece | 32,000 | `<pad>`, `</s>` |

### Tokens Spéciaux

Les tokenizers réels incluent des tokens spéciaux :

```python
special_tokens = {
    '<PAD>': 0,      # Padding (pour batch)
    '<UNK>': 1,      # Unknown token
    '<BOS>': 2,      # Beginning of sequence
    '<EOS>': 3,      # End of sequence
    '<MASK>': 4,     # Masking (BERT)
}
```

### Exemple avec GPT-2 Tokenizer

```python
# Utiliser le vrai tokenizer GPT-2 (nécessite transformers)
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

text = "Hello, how are you?"
tokens = tokenizer.encode(text)
print(tokens)  # [15496, 11, 703, 389, 345, 30]

decoded = tokenizer.decode(tokens)
print(decoded)  # "Hello, how are you?"
```

---

## 💡 Concepts Importants

### 1. Vocabulaire et Taille

**Trade-off** :
- **Petit vocabulaire** (5k tokens)
  - ✅ Moins de paramètres
  - ❌ Séquences plus longues
  - ❌ Moins expressif

- **Grand vocabulaire** (100k tokens)
  - ✅ Séquences plus courtes
  - ✅ Plus expressif
  - ❌ Plus de paramètres
  - ❌ Plus de mémoire

**Sweet spot** : 30k-50k tokens

### 2. Out-of-Vocabulary (OOV)

**Problème** : Que faire avec un mot jamais vu ?

```python
# Word-level tokenizer
vocab = {"chat": 0, "chien": 1, "oiseau": 2}
text = "hamster"  # ❌ Pas dans le vocab !

# Solution 1: Token <UNK>
encode("hamster") → [3]  # ID de <UNK>

# Solution 2: BPE (meilleur !)
encode("hamster") → ["ham", "ster"] → [456, 789]
# Pas de OOV car décomposé en sous-mots connus
```

### 3. Whitespace et Ponctuation

**Important** : Comment gérer les espaces ?

```python
# GPT-2 utilise des "Ġ" pour les espaces
"Hello world" → ["Hello", "Ġworld"]
#                         ↑ Espace préservé

# Permet de reconstruire exactement le texte
```

---

## 🎯 Impact de la Tokenization sur les LLMs

### 1. Longueur de Séquence

```python
text = "L'intelligence artificielle est fascinante"

# Character-level (43 tokens)
chars = list(text)  # ['L', "'", 'i', 'n', 't', ...]

# Word-level (4 tokens)
words = text.split()  # ["L'intelligence", "artificielle", "est", "fascinante"]

# BPE (6-8 tokens)
bpe = ["L", "'", "intelligence", "artific", "ielle", "est", "fasc", "inante"]
```

**Impact** :
- Plus de tokens = Plus de calcul
- Context window limité (ex: 2048 tokens pour GPT-3)

### 2. Multilinguisme

**Problème** : Langues non-latines

```python
# Anglais (efficace)
"Hello" → 1 token

# Chinois (moins efficace avec BPE traditionnel)
"你好" → 2-4 tokens (selon le tokenizer)

# Solution: SentencePiece (utilisé par mT5, XLM-R)
# Gère mieux toutes les langues
```

### 3. Code et Symboles

```python
code = "def hello(): print('Hi')"

# BPE peut mal découper le code
# → Tokenizers spécialisés pour le code (CodeGen, CodeT5)
```

---

## 🔬 Exercice Pratique : Analyser un Texte

In [ ]:
# Texte à analyser
long_text = """
L'intelligence artificielle transforme notre monde. 
Les modèles de langage comme GPT-4 peuvent comprendre et générer du texte.
"""

print("📝 Texte original:")
print(long_text)
print(f"\nLongueur: {len(long_text)} caractères\n")

# Créer un corpus simple
words = long_text.lower().split()
corpus_words = [word.strip('.,!?') for word in words if word.strip('.,!?')]

print(f"📚 Corpus: {len(corpus_words)} mots")

# Entraîner un tokenizer
small_tokenizer = SimpleBPETokenizer(vocab_size=50)
small_tokenizer.train(corpus_words, verbose=False)

print(f"\n🔤 Vocabulaire: {len(small_tokenizer.vocab)} tokens")

# Encoder le texte
encoded = small_tokenizer.encode(long_text.lower())
print(f"\n🔢 Encodé: {len(encoded)} tokens")
print(f"   Compression: {len(long_text) / len(encoded):.1f}× (caractères/tokens)")

# Afficher quelques tokens
print(f"\n📊 Premiers tokens:")
for i, tid in enumerate(encoded[:15]):
    token = small_tokenizer.inverse_vocab.get(tid, '<UNK>')
    print(f"   {i}: '{token}' (ID: {tid})")

---

## 🎓 Résumé et Concepts Clés

### Ce qu'on a appris

✅ **Pourquoi tokenizer** : Les réseaux ne comprennent que les nombres  
✅ **3 approches** : Character, Word, Subword (BPE)  
✅ **BPE** : Algorithme de compression itératif  
✅ **Implémentation** : Tokenizer BPE from scratch  
✅ **Encode/Decode** : Texte ↔ IDs  

### Concepts Clés

1. 🔑 **Token** : Unité de base (caractère, mot, ou sous-mot)
2. 🔑 **Vocabulaire** : Ensemble de tous les tokens possibles
3. 🔑 **BPE** : Fusionne itérativement les paires fréquentes
4. 🔑 **OOV** : Out-of-Vocabulary, résolu par BPE
5. 🔑 **Trade-offs** : Taille vocab vs longueur séquence

### Impact sur les LLMs

- Le tokenizer détermine **comment le texte est représenté**
- Impacte la **longueur des séquences** (donc le coût)
- Affecte le **multilinguisme** et la **gestion du code**
- GPT utilise **BPE avec ~50k tokens**

---

## 🚀 Prochaine Étape

**Notebook 02 - Embeddings** 🎯

Maintenant qu'on sait transformer du texte en IDs, on va apprendre à transformer ces IDs en **vecteurs riches** !

```
Texte → Tokens → IDs → EMBEDDINGS (vecteurs denses)
                        ↑ Prochain notebook !
```

**Pourquoi c'est important ?**
- Les IDs sont juste des nombres arbitraires (145, 298, 1023)
- Les embeddings capturent la **sémantique** (sens)
- Mots similaires → vecteurs proches

---

## 🎯 Exercices (Optionnels)

1. **Modifier le tokenizer** : Ajouter des tokens spéciaux `<PAD>`, `<UNK>`, `<BOS>`, `<EOS>`
2. **Corpus plus grand** : Entraîner sur un fichier texte complet
3. **Comparer** : Implémenter un tokenizer Word-level et comparer avec BPE
4. **Visualiser** : Créer un graphe montrant comment BPE merge les tokens
5. **Explorer** : Utiliser `transformers` library pour essayer le vrai GPT-2 tokenizer

---

**Prêt pour les embeddings ? Let's go ! 🚀**

**→ Notebook suivant : `02_embeddings.ipynb`**